# Introducción


## Objetivo

Utilizar Llama 2.0, Langchain y ChromaDB para crear un sistema de Generación con Recuperación Mejorada (RAG). Esto nos permitirá hacer preguntas sobre nuestros documentos (que no se incluyeron en los datos de entrenamiento), sin necesidad de ajustar finamente el Modelo de Lenguaje Grande (LLM, por sus siglas en inglés).
Cuando se utiliza RAG, si se plantea una pregunta, primero se realiza un paso de recuperación para obtener documentos relevantes de una base de datos especial, una base de datos vectorial donde se indexaron estos documentos.

## Definiciones

* LLM - Modelo de Lenguaje Grande (Large Language Model)
* Llama 2.0 - LLM de Meta
* Langchain - un marco diseñado para simplificar la creación de aplicaciones utilizando LLM
* Base de datos vectorial - una base de datos que organiza datos a través de vectores de alta dimensión
* ChromaDB - base de datos vectorial
* RAG - Generación con Recuperación Mejorada (consulte más detalles sobre RAG a continuación)

## Detalles del modelo

* **Modelo**:  Mistral
* **Variante**: 7b-v0.1-hf (7b: 7B dimm. hf: HuggingFace build)
* **Versión**: V1
* **Framework**: PyTorch



## ¿Qué es un sistema de Generación con Recuperación Mejorada (RAG)?

Los Modelos de Lenguaje Grande (LLM) han demostrado su capacidad para comprender el contexto y proporcionar respuestas precisas a diversas tareas de Procesamiento de Lenguaje Natural (NLP), incluyendo la resumen, preguntas y respuestas, cuando se les solicita. Si bien son capaces de proporcionar respuestas muy buenas a preguntas sobre información con la que fueron entrenados, tienden a alucinar cuando el tema trata sobre información que "no saben", es decir, no estaba incluida en sus datos de entrenamiento. La Generación con Recuperación Mejorada combina recursos externos con LLM. Por lo tanto, los dos componentes principales de un sistema RAG son un recuperador y un generador.

La parte del recuperador se puede describir como un sistema que es capaz de codificar nuestros datos para que se puedan recuperar fácilmente las partes relevantes al consultarlos. La codificación se realiza utilizando incrustaciones de texto, es decir, un modelo entrenado para crear una representación vectorial de la información. La mejor opción para implementar un recuperador es una base de datos vectoriales. Como bases de datos vectoriales, existen múltiples opciones, tanto productos de código abierto como comerciales. Algunos ejemplos son ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Nuestra opción en este cuaderno será una instancia local de ChromaDB (persistente).

Para la parte del generador, la opción más obvia es un LLM. En este cuaderno utilizaremos un modelo LLaMA v2 cuantificado, de la colección de Modelos de Kaggle.

La orquestación del recuperador y el generador se realizará utilizando Langchain. Una función especializada de Langchain nos permite crear el recuperador-generador en una sola línea de código.


# Installations, imports, utils

In [1]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.3 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from time import time

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Inicializar modelo, tokenizador, y canal de consultas.


Preparar el modelo y el tokenizador.

In [3]:
time_1 = time()


model_name = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prepare model, tokenizer: 139.717 sec.


Definir el query pipeline

In [4]:
time_1 = time()
query_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.0 sec.


In [5]:
messages = [
    {"role": "user", "content": "Cual es la capital de Alemania?"},
]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [6]:
print(decoded[0])

<s> [INST] Cual es la capital de Alemania? [/INST] La capital de Alemania es Berlín.</s>


Credit:
 - Adapted from https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

## Testear la query pipeline

Testeamos el pipeline con una query sobre...

In [7]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [8]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 1.593 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.


In [9]:
test_model(tokenizer,
           query_pipeline,
           "Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 10.749 sec.
Result: Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.

En este discurso sobre el estado de la nación EEUU, Bush expone los temas críticos de la sociedad estadounidense, incluyendo la economía, la seguridad nacional y la educación pública. El Presidente advierte sobre los desafíos económicos del siglo XXI, como el crecimiento del desempleo y la deuda, y propone una serie de medidas para promover el crecimiento económico y la competitividad. También discute la amenaza terrorista y la seguridad nacional, y reitera los Estados Unidos se comprometen a defender la paz mundial. El Presidente también señala que el sistema educativo de los Estados Unidos no
